In [7]:
import requests
from urllib import parse
from bs4 import BeautifulSoup
import pymongo
import time
import random
import pandas as pd

In [2]:
# mongoDB 연결 and crawling_review collection 생성
conn = pymongo.MongoClient()
db = conn.crawling_review

In [4]:
# 네이버 평점 url, 여러 페이지 조회하기 위해 url page= 부분 format
base_url = "https://movie.naver.com/movie/point/af/list.naver?&page={}"

data_list = []

In [5]:
# 평점 크롤링 
for page in range(1,1000):
    url = base_url.format(page)
    res = requests.get(url)
    # 200 : 요청을 제대로 처리햇다는 뜻
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        html_path = soup.select('td.title')
        for td in html_path:
            # .text로 조회하면 태그들도 포함되므로 br태그 그 태그 내용 찾기위해 next_sibling
            review = td.select_one('br').next_sibling.text.strip()
            star = td.select_one('div.list_netizen_score > em').text.strip()
            data_list.append((star,review))
        delay = round(random.uniform(0.2,1.2),2)
        time.sleep(delay)
print('완료')

완료


In [47]:
star = data['평점'].values
star = list(star)
st_db = []
for s in star:
    st_db.append(s)
    
comment = data['리뷰'].values
comment = list(comment)
comm_db = []
for c in comment:
    comm_db.append(c)

In [50]:
# mongoDB 저장
doc = {
    'star':st_db,
    'comment':comm_db
}
db.movies.insert_one(doc)

In [53]:
# DB 연결 종료
conn.close()

In [52]:
# csv 저장
data.to_csv('movie_review.csv')